# Busca (Parte II)

## Mochila com Número Irrestrito de Itens com Valor

In [1]:
max_size = 19
items = [(1,3),(4,6),(5,7)]

Tamanho de Estados

In [2]:
def state_size(state, items):
    size = 0
    for i in range(len(state)):
        size += state[i] * items[i][1]
    return size

In [3]:
state_size([1,1,1], items)

16

Avaliação de Estados

In [4]:
def evaluate_state(state, items):
    value = 0
    for i in range(len(state)):
        value += state[i] * items[i][0]
    return value

In [5]:
evaluate_state([1,2,0], items)

9

Geração de Estados

In [6]:
def generate_states(initial_state):
    states = []
    for i in range (len(initial_state)):
        aux = initial_state.copy()
        aux[i] = initial_state[i] + 1
        states.append(aux)
    return states

In [7]:
generate_states([0,0,0])

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]

Roleta

Soma de valores de todos os estados

In [8]:
def states_total_value(states):
    total_sum = 0
    for state in states:
        total_sum = total_sum + state[0]
    return total_sum

In [9]:
states_total_value([(1, [1,0,0]), (4, [0, 1, 0]), (5, [0, 0, 1])])

10

Construção da roleta

In [10]:
def roulette_construction(states):
    aux_states = []
    roulette = []
    total_value = states_total_value(states)

    for state in states:
        value = state[0]
        if total_value != 0:
            ratio = value/total_value
        else:
            ratio = 1
        aux_states.append((ratio,state[1]))
 
    acc_value = 0
    for state in aux_states:
        acc_value = acc_value + state[0]
        s = (acc_value,state[1])
        roulette.append(s)
    return roulette

In [11]:
roulette_construction([(1, [1,0,0]), (4, [0, 1, 0]), (5, [0, 0, 1])])

[(0.1, [1, 0, 0]), (0.5, [0, 1, 0]), (1.0, [0, 0, 1])]

Rodar a roleta

In [12]:
import random

def roulette_run (rounds, roulette):
    if roulette == []:
        return []
    selected = []
    while len(selected) < rounds:
        r = random.uniform(0,1)
        for state in roulette:
            if r <= state[0]:
                selected.append(state[1])
                break
    return selected

In [13]:
roulette_run (1, [(0.1, [1, 0, 0]), (0.5, [0, 1, 0]), (1.0, [0, 0, 1])])

[[0, 0, 1]]

Método Heurístico - Construtivo

Hill Climbing Não Determinístico

In [14]:
import time

def hill_climbing(max_size, items, max_time):
    start = time.process_time()
    current_state = [0]*len(items)
    optimal_value = 0
    optimal_size = 0
    optimal_state = current_state
    valid_states = len(items)
    end = 0

    while valid_states != 0 and end-start <= max_time:
        valid_states = len(items)
        possible_states = generate_states(current_state)
        
        states = []
        for state in possible_states:
            aux_val = evaluate_state(state, items)
            aux_size = state_size(state, items)
            if aux_size <= max_size:
                states.append((aux_val, state))
                if aux_val > optimal_value:
                    optimal_value = aux_val
                    optimal_size = aux_size
                    optimal_state = state
            else:
                valid_states = valid_states - 1                
        
        if valid_states > 0:
            current_state = roulette_run (1, roulette_construction(states))[0]

        end = time.process_time()

    return optimal_state, optimal_size, optimal_value

In [15]:
hill_climbing(max_size, items, 120)

([1, 0, 2], 17, 11)

Métodos Heurísticos - Busca Local

Estado Inicial Aleatório

In [16]:
import random

def generate_initial_state(max_size, items):
    initial_state = []
    for i in range(len(items)):
        initial_state.append(random.randint(0,max_size//items[i][1]))
    print (initial_state)
    if state_size(initial_state,items) > max_size:
        while True:
            for i in range(len(items)):
                if initial_state[i] > 0:
                    initial_state[i] = initial_state[i] - 1
                    if state_size(initial_state,items) <= max_size:
                        return initial_state
    return initial_state

In [17]:
generate_initial_state(1381, [(1,3),(4,6),(5,7),(3,4),(2,6),(2,3),(6,8),(1,2),(3,5),(7,10),(10,15),(13,20),(24,25),(29,37)])

[300, 163, 15, 34, 22, 10, 112, 311, 95, 47, 56, 47, 5, 27]


[193, 57, 0, 0, 0, 0, 6, 205, 0, 0, 0, 0, 0, 0]

In [18]:
generate_initial_state(max_size, items)

[6, 3, 2]


[4, 1, 0]

Operador de Mudança de Estado

In [19]:
def change_state(state,position,value):
    if state[position] == 0 and value < 0:
        return []
    state[position] = state[position] + value
    return state

In [20]:
change_state([2,0,1], 2, -1)

[2, 0, 0]

Vizinhança

In [21]:
def generate_neighborhood(max_size, items, state):
    neighborhood = []
    for i in range(len(state)):
        aux = state.copy()
        new_state = change_state(aux,i,1)
        if state_size (new_state, items) <= max_size:
            neighborhood.append(new_state)
    for i in range(len(state)):
        aux = state.copy()
        new_state = change_state(aux,i,-1)
        if new_state != [] and state_size (new_state, items) <= max_size:
            neighborhood.append(new_state)
    return neighborhood

In [22]:
generate_neighborhood(max_size, items, [0,2,1])

[[0, 1, 1], [0, 2, 0]]

Gradiente de Subida

In [23]:
def gradient_ascent(solution,max_size,items):
    max_value = evaluate_state(solution,items)
    better = True
    while better:       
        neighborhood = generate_neighborhood(max_size, items, solution)
        better = False
        for s in neighborhood:
            value = evaluate_state(s,items)
            if value > max_value and state_size(s,items) <= max_size:
                solution = s
                max_value = value
                better = True             
    return solution, state_size(solution,items), max_value

In [24]:
gradient_ascent([1,0,2],max_size,items)

([1, 0, 2], 17, 11)

In [25]:
gradient_ascent(generate_initial_state(max_size, items),max_size,items)

[0, 1, 2]


([1, 0, 2], 17, 11)

Gradiente de Subida Recursivo

In [26]:
def gradient_ascent_rec(solution,max_size,items):
    max_value = evaluate_state(solution,items)
    neighborhood = generate_neighborhood(max_size, items, solution)
    better = False
    for s in neighborhood:
        value = evaluate_state(s,items)
        if value > max_value and state_size(s,items) <= max_size:
            solution = s
            max_value = value
            better = True        
    if(better):
        return gradient_ascent(solution,max_size,items)
    else:
        return solution, state_size(solution,items), max_value

In [27]:
gradient_ascent_rec([1,0,2],max_size,items)

([1, 0, 2], 17, 11)

In [28]:
gradient_ascent_rec(generate_initial_state(max_size, items),max_size,items)

[1, 1, 2]


([1, 0, 2], 17, 11)

Primeiro elemento de uma lista ou tupla

In [29]:
def first(x):
    return x[0]

Vizinhança ordenada decrescentemente por valor

In [30]:
def ordered_neighborhood(solution,max_size,items):
    neighborhood = generate_neighborhood(max_size, items, solution)
    sv = []
    for s in neighborhood:
        v = evaluate_state(s,items)
        sv.append ((v,s))
    sv = sorted (sv, key = first, reverse = True) 
    return sv

In [31]:
ordered_neighborhood([0,2,1],max_size,items)

[(9, [0, 1, 1]), (8, [0, 2, 0])]

Atualiza lista tabu

In [32]:
def update_tabu (tabu, s, tabu_size):
    if len(tabu) == tabu_size:
        tabu = tabu[1:]
    tabu = tabu + [s]
    return tabu

In [33]:
update_tabu([[0,0,0], [1,3,1], [2,1,3]], [1,3,2], 3)

[[1, 3, 1], [2, 1, 3], [1, 3, 2]]

Busca Tabu

In [34]:
import time

def tabu_search(init,max_size,items,max_time,max_non_improv,tabu_size):
    start = time.process_time()
    current = init
    tabu = []
    non_improv = 0
    end = 0
    
    if state_size(init,items) <= max_size:    
        best = init
        best_value = evaluate_state(init,items)
    else:
        best = [0]*len(items)
        best_value = 0   
    
    while non_improv <= max_non_improv and end-start <= max_time:       
        onbh = ordered_neighborhood(current, max_size, items)
        #print(onbh)

        while onbh != []:
            v, s = onbh[0]
            
            if v > best_value:
                best = s
                best_value = v
                non_improv = 0 
            else:
                non_improv = non_improv + 1
                
            if not s in tabu:
                current = s
                tabu = update_tabu (tabu,s,tabu_size)
                break
            else:
                onbh = onbh[1:] 
        
        end = time.process_time()

        if onbh == []:
            break
    
    return best, state_size(best,items), best_value

In [35]:
init = generate_initial_state(max_size, items)
print(init)
tabu_search(init,max_size,items,120,5,3)

[2, 3, 2]
[0, 2, 1]


([0, 2, 1], 19, 13)

Probabilidade de aceitar mudança para pior estado

In [36]:
import math
import random

def change_probability(value,best_value,t):
    p = 1/(math.exp(1)**((best_value-value)/t))
    r = random.uniform(0,1)
    if r < p:
        return True
    else:
        return False

In [37]:
change_probability(10, 13, 200)

True

Seleção aleatória de elemento de lista

In [38]:
def random_state(states):
    index = random.randint(0,len(states)-1)
    return states[index]

In [39]:
random_state([[0,0,0],[0,2,1],[0,1,1]])

[0, 0, 0]

Recozimento Simulado

In [40]:
import time

def simulated_annealing(state,t,alfa,items,max_size,iter_max,max_time):
    solution = state
    max_value = evaluate_state(solution,items)
    start = time.process_time()
    end = 0
    
    while t >= 1 and end-start <= max_time:        
        
        for _ in range(iter_max):    
            neighborhood = generate_neighborhood(max_size,items,state)
            if neighborhood == []:
                return solution,max_value,state_size(solution,values)                
            aux = random_state(neighborhood)
            aux_value = evaluate_state(aux,items)
            aux_size = state_size(aux,items)
            state_value = evaluate_state(state,items)
            if aux_value > state_value and aux_size <= max_size:
                state = aux
                if aux_value > max_value:
                    solution = aux
                    max_value = aux_value
            else:
                if aux_size <= max_size:
                    if change_probability(aux_value,state_value,t):
                        state = aux
        t = t*alfa
        end = time.process_time()

    return solution, state_size(solution,items), max_value

In [41]:
init = generate_initial_state(max_size, items)
print(init)
simulated_annealing(init,200,0.1,items,max_size,10,120)

[5, 3, 2]
[3, 1, 0]


([2, 1, 1], 19, 11)

GRASP

In [42]:
def grasp(max_size,items,iter_max,max_time):
    optimal = [0]*len(items)
    optimal_value = 0
    start = time.process_time()
    end = 0
    iter = 0
    while iter < iter_max and end-start <= max_time:
        iter += 1
        s = hill_climbing(max_size, items, max_time//iter_max)[0]
        s = gradient_ascent(s,max_size,items)[0]
        s_value = evaluate_state(s, items)
        if s_value > optimal_value:
            optimal = s
            optimal_value = s_value
        end = time.process_time()
    return optimal,state_size(optimal,items),optimal_value

In [43]:
grasp(max_size,items,10,120)

([0, 2, 1], 19, 13)

### Exercícios de Fixação

1. Explique como metaheurísticas de busca local conseguem escapar de vales e picos não globais? 

2. A metaheurística multistart consiste em gerar várias soluções iniciais aleatórias e executar um algoritmo de busca local (como, por exemplo, o gradiente de subida) para cada uma delas. O melhor resultado encontrado nas várias buscas é o escolhido. Caracterize o que é a etapa de diversificação e o que é a etapa de intensificação nesta meta-heurística. 

3. Alterar o algoritmo de gradiente de subida para usar uma roleta ao escolher na vizinhança o próximo estado a ser considerado. Note que o critério de parada também deve ser modificado. Como sugestão de critério de parada considere estabelecer um número máximo de vezes que se escolhe uma solução que não melhora a melhor solução encontrada até o momento.

4. Explique porque um tamanho grande na lista tabu pode provocar a interrupção prematura da busca?

5. Qual a importância da temperatura no método de recozimento simulado para regular as etapas de diversificação e intensificação da busca? 

6. Porque o algoritmo construtivo guloso usado no GRASP necessita ser não determinístico?

7. Ajuste os hiperparâmetros de tabu search, resfriamento simulado e grasp para fazer com que eles encontrem o ótimo global do problema usado nesta aula na grande maioria de execuções. 

8. Considere o problema da mochila irrestrita com tamanho de mochila igual a 1381 e com a seguinte lista de pares (valor, tamanho) de itens {(1,3),(4,6),(5,7),(3,4),(2,6),(2,3),(6,8),(1,2),(3,5),(7,10),(10,15),(13,20),(24,25),(29,37)}. Execute várias vezes os métodos da questão 7 com os melhores valores de hiperaprâmetros encontrados por você ao resolver aquela questão. Reporte o melhor resultado encontrado, qual(is) o(s) método(s) que o acharam e qual o percentual das execuções do método encontrou o melhor resultado. 

9. Altere a implementação do método de recozimento simulado para que ele também interrompa a busca quando o algoritmo avaliar um determinado número de novos estados sem mudar a melhor solução encontrada até o momento. 

10. Altere a implementação do método grasp para usar o método tabu search na etapa de busca local.